## 🧭 Project Purpose

This mini-project demonstrates **Retrieval-Augmented Generation (RAG)** versus **No-RAG** using content collected from the **FAIR Data Innovations Hub** website.

### What we will build:

- A simple pipeline that **downloads ~20 web pages** from `fairdataihub.org` and saves them as plain text.
- A **vector index (FAISS)** built from those texts using **Ollama embeddings**.
- Two answering modes for the **same questions**:
  - **No-RAG:** the LLM answers using only its general knowledge.
  - **RAG:** the LLM answers using **retrieved website context**, improving grounding and reducing hallucinations.

### Why this matters

Website content changes and includes specific names, projects, and descriptions.  
RAG makes answers more **document-based** and **verifiable**, especially for details that a model may otherwise guess.


## 🔧 Step 1 — Install Python dependencies (run once)
! pip install -r requirements.txt


## 🦙 Step 2 — Configure Ollama models (LLM + Embeddings)

This step initializes two local Ollama components:

- **LLM (generation model)**: used to produce answers.
- **Embeddings model**: used to convert text chunks into vectors for retrieval (FAISS).

**Inputs**
- `LLM_MODEL = "llama3.2"`
- `EMBED_MODEL = "nomic-embed-text"`

**Outputs**
- `llm`: a `ChatOllama` instance for text generation
- `embeddings`: an `OllamaEmbeddings` instance for vector embeddings

**Prerequisite**
- The models must be available locally (pulled via Ollama), e.g.:
  - `ollama pull llama3.2`
  - `ollama pull nomic-embed-text`


In [34]:
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader


# --- Choose models you have pulled via `ollama pull ...`
LLM_MODEL = "llama3.2"            # generation model
EMBED_MODEL = "nomic-embed-text"  # embeddings model (recommended)

# Initialize Ollama chat model
llm = ChatOllama(model=LLM_MODEL, temperature=0.2)

# Initialize Ollama embeddings model
embeddings = OllamaEmbeddings(model=EMBED_MODEL)

print("✅ Ollama models configured:", LLM_MODEL, "|", EMBED_MODEL)

✅ Ollama models configured: llama3.2 | nomic-embed-text


## 🌐 Step 3 — Collect website content (crawl ~20 pages and save as `.txt`)

This step builds a small local corpus from the **FAIR Data Innovations Hub** website so the RAG system can retrieve information from it.

### Purpose

- Visit a limited number of pages starting from the homepage (`https://fairdataihub.org/`)
- Collect internal links using a simple breadth-first search (BFS)
- Download each page’s HTML
- Clean and extract readable text
- Save each page as a separate `.txt` file in `data/fairdata_texts/`

### Key settings

- **START_URL**: the crawl entry point (`https://fairdataihub.org/`)
- **MAX_PAGES = 20**: caps the number of pages to keep the dataset small and fast
- **MAX_DEPTH = 2**: limits how far the crawler follows links from the start page
- **OUT_DIR**: output folder for saved texts (`data/fairdata_texts/`)

### How the crawler works

1. **Initialize a web session**
   - Uses `requests.Session()` with a browser-like `User-Agent` for stable page fetching.

2. **Filter what to crawl**
   - Skips authentication/legal and irrelevant pages using `SKIP_TERMS` (e.g., login, privacy, cookie).
   - Skips non-text assets using `SKIP_EXT` (e.g., images, videos, zip files, PDFs).

3. **Discover pages (BFS)**
   - Uses a queue (`deque`) to explore internal links.
   - Only keeps links that stay inside `fairdataihub.org`.
   - Stops when it reaches `MAX_PAGES` or `MAX_DEPTH`.

4. **Convert HTML → clean text**
   - Removes noisy elements such as `script`, `style`, `iframe`, and `form`.
   - Removes common layout sections like `header`, `nav`, and `footer`.
   - Extracts visible text and normalizes whitespace for readability.

5. **Save outputs**
   - Writes each page to a numbered `.txt` file (safe filename via `slugify`).
   - Prepends the original page URL at the top of the text file for traceability.

### Output

After this step, you will have up to **20 text files** in:

- `data/fairdata_texts/`

These `.txt` files become the input documents for:
- chunking (text splitting)
- embedding (vectorization)
- retrieval (FAISS)


In [61]:
# Step 3: Crawl ~20 fairdataihub.org pages and save as .txt in data/fairdata_texts/

!pip -q install requests beautifulsoup4

import re
from collections import deque
from pathlib import Path
from urllib.parse import urljoin, urlparse

import requests
from bs4 import BeautifulSoup

START_URL = "https://fairdataihub.org/"
MAX_PAGES = 20
MAX_DEPTH = 2

OUT_DIR = Path("data/fairdata_texts")
OUT_DIR.mkdir(parents=True, exist_ok=True)

session = requests.Session()
session.headers.update({"User-Agent": "Mozilla/5.0 (fairdataihub-simple-rag)"})

SKIP_TERMS = ["login", "signin", "signup", "register", "account", "privacy", "terms", "cookie"]
SKIP_EXT = (".png", ".jpg", ".jpeg", ".gif", ".svg", ".webp", ".zip", ".mp4", ".pdf")

def normalize_url(u: str) -> str:
    return u.split("#", 1)[0].rstrip("/")

def is_fairdata_url(u: str) -> bool:
    return urlparse(u).netloc.endswith("fairdataihub.org")

def should_skip(u: str) -> bool:
    ul = u.lower()
    return any(t in ul for t in SKIP_TERMS) or ul.endswith(SKIP_EXT)

def slugify(u: str) -> str:
    p = urlparse(u).path.strip("/") or "home"
    p = re.sub(r"[^a-zA-Z0-9]+", "-", p).strip("-").lower()
    return p[:120] or "page"

def html_to_text(html: str) -> str:
    soup = BeautifulSoup(html, "html.parser")
    for tag in soup(["script", "style", "noscript", "svg", "iframe", "form"]):
        tag.decompose()
    for sel in ["header", "footer", "nav", "aside"]:
        for t in soup.find_all(sel):
            t.decompose()
    text = soup.get_text("\n", strip=True)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

# 1) Collect internal URLs (BFS)
visited = set()
queue = deque([(START_URL, 0)])
urls = []

while queue and len(urls) < MAX_PAGES:
    url, depth = queue.popleft()
    url = normalize_url(url)

    if url in visited or depth > MAX_DEPTH:
        continue
    if not is_fairdata_url(url) or should_skip(url):
        continue

    visited.add(url)
    urls.append(url)

    try:
        r = session.get(url, timeout=20)
        if r.status_code != 200:
            continue
        ctype = r.headers.get("content-type", "").lower()
        if "text/html" not in ctype:
            continue

        soup = BeautifulSoup(r.text, "html.parser")
        for a in soup.find_all("a", href=True):
            nxt = normalize_url(urljoin(url, a["href"]))
            if is_fairdata_url(nxt) and not should_skip(nxt):
                queue.append((nxt, depth + 1))
    except Exception:
        pass

print(f"✅ Collected {len(urls)} URLs")

# 2) Save each page as .txt
for i, url in enumerate(urls, start=1):
    try:
        r = session.get(url, timeout=20)
        if r.status_code != 200:
            continue
        txt = html_to_text(r.text)
        out = OUT_DIR / f"{i:02d}_{slugify(url)}.txt"
        out.write_text(f"URL: {url}\n\n{txt}\n", encoding="utf-8")
        print(f"[{i:02d}] Saved -> {out.name}")
    except Exception as e:
        print(f"⚠️ Failed {url}: {e}")

print("\n✅ Done. Texts saved in:", OUT_DIR.resolve())


✅ Collected 20 URLs
[01] Saved -> 01_home.txt
[02] Saved -> 02_projects.txt
[03] Saved -> 03_team.txt
[04] Saved -> 04_impact.txt
[05] Saved -> 05_gallery.txt
[06] Saved -> 06_blog.txt
[07] Saved -> 07_events.txt
[08] Saved -> 08_contact-us.txt
[09] Saved -> 09_sodaforsparc.txt
[10] Saved -> 10_aireadi.txt
[11] Saved -> 11_posters-science.txt
[12] Saved -> 12_eyeact.txt
[13] Saved -> 13_careers.txt
[14] Saved -> 14_codefair.txt
[15] Saved -> 15_dmp-chef.txt
[17] Saved -> 17_fair-biors.txt
[18] Saved -> 18_fairshare.txt
[19] Saved -> 19_knowmore.txt
[20] Saved -> 20_sparclink.txt

✅ Done. Texts saved in: C:\Users\Nahid\RAG_blog\data\fairdata_texts


## ✂️ Step 4 — Load `.txt` pages and split into chunks

This step prepares the website text for retrieval by converting raw page files into **manageable chunks**. Chunking is essential because retrieval works best when we search over smaller, semantically coherent pieces of text rather than entire pages.

### Purpose

- Load the saved website pages from `data/fairdata_texts/`
- Convert each `.txt` file into a LangChain `Document`
- Split documents into overlapping chunks suitable for embeddings + retrieval

### What happens in this step

1. **Read input files**
   - Finds all `.txt` files inside `data/fairdata_texts/`
   - If no files exist, it raises an error (meaning the crawl step didn’t run or saved elsewhere)

2. **Load as LangChain Documents**
   - Each `.txt` file becomes one `Document`
   - Adds a simple metadata field like `source_file` so we still know which page the text came from

3. **Split into chunks**
   - Uses a recursive text splitter to break long text into smaller pieces
   - Chunks overlap slightly so important information isn’t cut off between boundaries

### Key settings (chunking)

- **chunk_size = 1000 characters**  
  Keeps chunks reasonably small for embedding and prompt context.

- **chunk_overlap = 150 characters**  
  Preserves continuity across chunk boundaries.

- **separators = ["\\n\\n", "\\n", " ", ""]**  
  Prefers splitting at paragraph boundaries first, then lines, then spaces, and finally hard-splits as a last resort.

### Output

At the end of this step you have:

- `docs`: a list of page-level Documents
- `chunks`: a larger list of chunk-level Documents (the real unit used for embeddings + retrieval)

These `chunks` will be embedded and indexed in the next step.


In [53]:
# Step 4: Load saved .txt pages -> split into chunks

from pathlib import Path
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

!pip -q install langchain langchain-community

TXT_DIR = Path("data/fairdata_texts")
paths = sorted(TXT_DIR.glob("*.txt"))

if not paths:
    raise FileNotFoundError(f"No .txt files found in {TXT_DIR}. Run Step 1 first.")

# 1) Load documents
docs = []
for p in paths:
    d = TextLoader(str(p), encoding="utf-8").load()[0]
    d.metadata["source_file"] = p.name
    docs.append(d)

print(f"✅ Loaded {len(docs)} documents")

# 2) Split into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    separators=["\n\n", "\n", " ", ""],
)

chunks = text_splitter.split_documents(docs)

print("✅ Total chunks:", len(chunks))
print("Sample chunk source:", chunks[0].metadata.get("source_file"))
print("Sample chunk text (first 250 chars):")
print(chunks[0].page_content[:250])


✅ Loaded 19 documents
✅ Total chunks: 120
Sample chunk source: 01_home.txt
Sample chunk text (first 250 chars):
URL: https://fairdataihub.org



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 🧠 Step 5 — Embed chunks and build a FAISS vector index

This step converts our text chunks into **vector embeddings** and stores them in a **FAISS** index. This index is what enables fast similarity search during RAG (i.e., retrieving the most relevant chunks for a user question).

### Purpose

- Connect to the local **Ollama** server
- Use an **embedding model** to vectorize each chunk
- Build a **FAISS vectorstore** for retrieval
- (Optional) save the FAISS index to disk for reuse

### What happens in this step

1. **Install FAISS**
   - Ensures FAISS is available to store and search embeddings efficiently.

2. **Verify Ollama is running**
   - Checks that the Ollama server is reachable at `http://localhost:11434`.
   - If not reachable, retrieval indexing cannot proceed (because embeddings cannot be generated).

3. **Create embeddings client**
   - Initializes an embeddings interface using the selected Ollama embedding model.

4. **Build the FAISS vectorstore**
   - Embeds every chunk
   - Inserts all vectors into a FAISS index
   - Produces a `vectorstore` object that supports similarity search (retrieval)

5. **Save the index (optional)**
   - Writes the FAISS index to a folder (e.g., `faiss_index_fairdata/`)
   - Useful to avoid rebuilding embeddings each time you restart the notebook

### Key settings

- **BASE_URL**: location of the Ollama server (default: `http://localhost:11434`)
- **EMBED_MODEL**: embedding model name (e.g., `nomic-embed-text` or `mxbai-embed-large`)
- **INDEX_DIR**: folder name used to persist the FAISS index

### Output

At the end of this step you have:

- `vectorstore`: a FAISS-backed index containing all embedded chunks  
- A saved index folder (optional): `faiss_index_fairdata/`

This `vectorstore` is used in the next step to retrieve relevant context for RAG answers.


In [55]:
# Step 5: Build FAISS vectorstore from chunks using Ollama embeddings

!pip -q install faiss-cpu

from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
import requests

BASE_URL = "http://localhost:11434"
EMBED_MODEL = "nomic-embed-text"  # or "mxbai-embed-large"

# Optional: check Ollama is reachable
try:
    r = requests.get(f"{BASE_URL}/api/tags", timeout=5)
    print("✅ Ollama reachable.")
except Exception as e:
    raise RuntimeError(
        f"Cannot reach Ollama at {BASE_URL}. "
        "Make sure Ollama is running (ollama serve). "
        f"Error: {e}"
    )

embeddings = OllamaEmbeddings(model=EMBED_MODEL, base_url=BASE_URL)

vectorstore = FAISS.from_documents(chunks, embeddings)

# Optional: save locally
INDEX_DIR = "faiss_index_fairdata"
vectorstore.save_local(INDEX_DIR)

print(f"✅ FAISS index built and saved to ./{INDEX_DIR}")


✅ Ollama reachable.
✅ FAISS index built and saved to ./faiss_index_fairdata


## ✅ Step 6 — Compare No-RAG vs RAG question answering (no citations)

This step demonstrates the core idea of the project: answering the **same questions** with and without retrieval.

### Purpose

- **No-RAG mode:** the LLM answers using only its general knowledge (higher risk of missing details or hallucination).
- **RAG mode:** the system retrieves relevant chunks from our FAISS index and forces the LLM to answer using only that website-derived context.

### What happens in this step

1. **Initialize the local LLM**
   - Connects to the Ollama server and loads a chat model (e.g., `llama3.2`).
   - Uses a low temperature for more consistent answers.

2. **Define the No-RAG function**
   - Sends the question directly to the LLM.
   - Includes a safety instruction: if unsure, the model should say so instead of inventing facts.

3. **Define the RAG function**
   - Converts the FAISS `vectorstore` into a retriever (`k=5`).
   - Retrieves the top `k` most relevant chunks for the question.
   - Concatenates retrieved chunks into a short context block.
   - Prompts the LLM to answer using **only** the provided context.
   - If the answer is missing from the context, it must respond with:  
     “Not found in the provided pages.”

4. **Run a small evaluation set**
   - Executes a list of website-related questions.
   - Prints **No-RAG** and **RAG** answers side by side to show the difference.

### Why this demonstrates the power of RAG

RAG is strongest when questions require **site-specific facts**, such as:
- leadership or team member names,
- project titles and descriptions,
- tool names and their functions.

Without retrieval, the model may guess or omit details. With retrieval, the answer is grounded in the downloaded website text.


In [60]:
# Step 6: NO-RAG vs RAG answering 

try:
    from langchain_ollama import ChatOllama
except Exception:
    from langchain_community.chat_models import ChatOllama

BASE_URL = "http://localhost:11434"
LLM_MODEL = "llama3.2"

llm = ChatOllama(model=LLM_MODEL, base_url=BASE_URL, temperature=0.2)

def answer_without_rag(question: str) -> str:
    prompt = f"""Answer the question as best you can.

If you are not sure, say "I am not sure" and do NOT invent details.

Question:
{question}

Answer:
"""
    return llm.invoke(prompt).content

def answer_with_rag(question: str, k: int = 5) -> str:
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})
    retrieved = retriever.invoke(question)

    # keep context short to avoid overload
    context = "\n\n---\n\n".join(d.page_content for d in retrieved)

    prompt = f"""Answer the question using ONLY the website context below.
If the answer is not in the context, say: "Not found in the provided pages."

Website context:
{context}

Question:
{question}

Answer:
"""
    return llm.invoke(prompt).content


questions = [
    "What is FAIR Data Innovations Hub and what does it do?",
    "Who are the members of the FAIR Data Innovations Hub team?",
    "What is DMP chef and what does it do?",
    "List the main projects mentioned on the FAIR Data Innovations Hub website.",
]

for q in questions:
    print("\n" + "="*110)
    print("QUESTION:", q)

    print("\n--- NO RAG ---")
    print(answer_without_rag(q))

    print("\n--- WITH RAG ---")
    print(answer_with_rag(q, k=5))


QUESTION: What is FAIR Data Innovations Hub and what does it do?

--- NO RAG ---
I am not sure.

--- WITH RAG ---
The FAIR Data Innovations Hub is a platform that provides tools and resources to make FAIR (Findable, Accessible, Interoperable, Reusable) data sharing effortless. It includes a web platform with intuitive user interfaces and automation tools to help researchers manage, curate, and share FAIR, ethically-sourced, and AI-ready datasets.

QUESTION: Who are the members of the FAIR Data Innovations Hub team?

--- NO RAG ---
I am not sure who the specific members of the FAIR Data Innovations Hub team are.

--- WITH RAG ---
Bhavesh Patel
Dorian Portillo
Sanjay Soundarajan

QUESTION: What is DMP chef and what does it do?

--- NO RAG ---
I am not sure.

--- WITH RAG ---
DMP Chef is a web tool that helps researchers create compliant and machine-actionable Data Management Plans (DMPs) in minutes. It generates drafts of funder-compliant DMPs tailored to their grant proposals, even cre